In [1]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np

from sklearn.metrics import accuracy_score
from keras_core import utils
from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate
from aepy.models.base.default_architectures import (Decoder_Conv_MNIST, Decoder_MLP,
                                               Encoder_Conv_MNIST, Encoder_MLP)
from aepy.models.vae.vae_model import VAE
from aepy.pipelines.training import TrainingPipeline

Using TensorFlow backend


2023-11-22 14:07:45.345630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-11-22 14:07:49 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-11-22 14:07:49 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-11-22 14:07:49 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-11-22 14:07:49 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [3]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)

In [4]:
# Model creation
model = VAE(input_dim=(x_train.shape[0], x_train.shape[1]), latent_dim=10, downstream_task='classification',
            encoder=Encoder_Conv_MNIST, decoder=Decoder_Conv_MNIST, layers_conf=[32, 64], n_classes=n_classes)

2023-11-22 14:07:50 [INFO]: Classificator available for the latent space of the autoencoder


In [5]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=2)

trained_model = pipe(train_data=train_data)

2023-11-22 14:07:50 [INFO]: +++ training_pipeline +++
2023-11-22 14:07:50 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-11-22_14-07-50


Epoch 1/2


2023-11-22 14:07:51.355076: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


ValueError: in user code:

    File "/home/lucas/experiment/aepy/examples/../aepy/models/vae/vae_model.py", line 174, in train_step  *
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
    File "/home/lucas/python_envs/aepy-test/lib/python3.10/site-packages/keras_core/src/optimizers/base_optimizer.py", line 206, in apply_gradients  *
        self.apply(grads, trainable_variables)
    File "/home/lucas/python_envs/aepy-test/lib/python3.10/site-packages/keras_core/src/optimizers/base_optimizer.py", line 249, in apply  *
        grads, trainable_variables = self._filter_empty_gradients(
    File "/home/lucas/python_envs/aepy-test/lib/python3.10/site-packages/keras_core/src/optimizers/base_optimizer.py", line 425, in _filter_empty_gradients  *
        raise ValueError("No gradients provided for any variable.")

    ValueError: No gradients provided for any variable.


In [ ]:
y_hat = trained_model.predict(test_data)

evaluate(y_true=np.argmax(test_data['labels'], axis=1), 
         y_hat=np.argmax(y_hat['clf'], axis=1),
         sel_metric=accuracy_score)

In [8]:
import tensorflow as tf
tensor = tf.range(10)
tf.print(tensor, output_stream=sys.stderr)


[0 1 2 ... 7 8 9]
